In [5]:
import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import tensorflow as tf
from keras import models
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.models import Sequential
from tensorflow.keras import Model, layers
from sklearn.model_selection import train_test_split
from sklearn import  metrics
from keras.applications.vgg16 import VGG16
from keras import optimizers
from tensorflow.keras.datasets import fashion_mnist
from keras.utils import np_utils

In [6]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [7]:
x_train=tf.image.grayscale_to_rgb(tf.expand_dims(x_train, axis=3)) 
x_test=tf.image.grayscale_to_rgb(tf.expand_dims(x_test, axis=3))

In [8]:
x_train=tf.image.resize_with_pad(x_train,48,48)
x_test=tf.image.resize_with_pad(x_test,48,48)

x_train /= 255
x_test  /= 255

In [9]:
input_shape = (48, 48, 3)
batch_size = 16
num_classes = 10
epochs = 5 

y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

In [10]:
conv_base = VGG16(weights="imagenet", include_top=False, input_shape = input_shape)
conv_base.trainable = False

58889256/58889256 [==============================] - 4s 0us/step


In [11]:
model = models.Sequential([
   conv_base,
   layers.Flatten(),
   layers.Dense(128, activation="relu"),
   layers.Dropout(0.5),
   layers.Dense(num_classes, activation='softmax'),
])

model.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [12]:
history = model.fit(x_train, y_train, batch_size=batch_size, 
                    epochs=epochs, verbose=1, validation_data=(x_test, y_test))
accuracy = model.evaluate(x_test, y_test, verbose=0)

Epoch 1/5
3750/3750 [==============================] - 1224s 326ms/step - loss: 0.6030 - accuracy: 0.7884 - val_loss: 0.4260 - val_accuracy: 0.8430
Epoch 2/5
3750/3750 [==============================] - 1248s 333ms/step - loss: 0.4580 - accuracy: 0.8340 - val_loss: 0.4037 - val_accuracy: 0.8506
Epoch 3/5
3750/3750 [==============================] - 1222s 326ms/step - loss: 0.4263 - accuracy: 0.8450 - val_loss: 0.3853 - val_accuracy: 0.8571
Epoch 4/5
3750/3750 [==============================] - 1199s 320ms/step - loss: 0.4109 - accuracy: 0.8502 - val_loss: 0.3731 - val_accuracy: 0.8661
Epoch 5/5
3750/3750 [==============================] - 1177s 314ms/step - loss: 0.3981 - accuracy: 0.8552 - val_loss: 0.3757 - val_accuracy: 0.8636


In [13]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 24, 24, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 12, 12, 128)       0     

In [14]:
conv_base = VGG16(weights="imagenet", include_top=False, input_shape = input_shape)

conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
    if layer.name == "block5_conv1":
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

modified_model_vgg16 = models.Sequential([
   conv_base,
   layers.Flatten(),
   layers.Dense(256, activation="relu"),
   layers.Dropout(0.5),
   layers.Dense(num_classes, activation='softmax'),
])

modified_model_vgg16.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [16]:
history = modified_model_vgg16.fit(x_train, y_train, batch_size=32, steps_per_epoch=50,
                                    epochs=30, verbose=1, validation_data=(x_test, y_test), validation_steps=50)

Epoch 1/30
50/50 [==============================] - 72s 1s/step - loss: 0.5081 - accuracy: 0.8313 - val_loss: 0.3862 - val_accuracy: 0.8650
Epoch 2/30
50/50 [==============================] - 74s 1s/step - loss: 0.4097 - accuracy: 0.8700 - val_loss: 0.3515 - val_accuracy: 0.8737
Epoch 3/30
50/50 [==============================] - 88s 2s/step - loss: 0.4431 - accuracy: 0.8481 - val_loss: 0.3859 - val_accuracy: 0.8644
Epoch 4/30
50/50 [==============================] - 91s 2s/step - loss: 0.4140 - accuracy: 0.8694 - val_loss: 0.3589 - val_accuracy: 0.8606
Epoch 5/30
50/50 [==============================] - 74s 1s/step - loss: 0.3771 - accuracy: 0.8838 - val_loss: 0.3554 - val_accuracy: 0.8725
Epoch 6/30
50/50 [==============================] - 73s 1s/step - loss: 0.4121 - accuracy: 0.8650 - val_loss: 0.4202 - val_accuracy: 0.8438
Epoch 7/30
50/50 [==============================] - 73s 1s/step - loss: 0.3810 - accuracy: 0.8675 - val_loss: 0.3770 - val_accuracy: 0.8644
Epoch 8/30
50/50 [==

In [17]:
accuracy = modified_model_vgg16.evaluate(x_test, y_test, verbose=0)

print(f'Accuracy:', accuracy[1])

Accuracy: 0.8925999999046326
